In [1]:
# Apdoroja Algirdo forensics.csv  

# Planas
# 1. Nuskaitome failą su statistika apie nesutampančius rpeaks 'information_about_rpeaks_2552.csv',
#   gautą su '7_3_zive_rpeaks_misalignments_org.ipynb'
#   Iš jo suformuojame failus su EKG failų vardais: vieną, kur rpeaks sutampa, ir kitą - kur nesutampa
# 
# 2. Nuskaitome failą su statistika apie koreguotus rpeaks 'forensics.csv, gautus iš Algirdo'
#    Iš jo suformuojame failus su EKG failų vardais: vieną, kur rpeaks koreguoti, kitą - kur rpeaks nejudinti  
# 
# 3. Sulyginame failus vieną su kitais, surasdami skirtumus
# 4. Suformuojame nesutampančių sąrašą


import neurokit2 as nk
import pandas as pd
import numpy as np
from pathlib import Path
import os, sys, json
from zive_util_vu import zive_read_df_rpeaks, get_rec_file_name, split_SubjCode, get_recId, zive_read_file_1ch


pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'
# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows, GUNDAS
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, HERKULIS
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'DUOM_2022_RUDUO_ATRANKA'

db_path = Path(Duomenu_aplankas, db_folder)
rec_dir = Path(db_path, 'InformacijaApieRpeaks')

print("\nPASIRUOŠIMAS")

# 1. Nuskaitome failą su statistika apie nesutampančius rpeaks 'information_about_rpeaks_2552.csv',
#   gautą su '7_3_zive_rpeaks_misalignments_org.ipynb'
#   Iš jo suformuojame failus su EKG failų vardais: vieną, kur rpeaks sutampa, ir kitą - kur nesutampa
filename_about_rpeaks_coinc = 'information_about_rpeaks_2552.csv'
print("\nInformacija iš failo:", filename_about_rpeaks_coinc)
file_path = Path(rec_dir, filename_about_rpeaks_coinc)
df_about_coinc = pd.read_csv(file_path, dtype={"ind":int, "filename":"string"})
print(f"Įrašų faile {filename_about_rpeaks_coinc}: {len(df_about_coinc)}")
# print(df_coinc.head())
# Suformuojame atitinkamą failų vardų sąrašą
lst_file_names_all = list(df_about_coinc["filename"].astype(str))
print(f"Faile yra visų anotuotų įrašų sąrašas. Jų kiekis: {len(lst_file_names_all)}")

# Surenkame eilutes, kurios atitinka nesutampančius rpeak
# naudojame požymius Flagged as Deleted == Yes arba Has Deviated == Yes
df_rpeaks_not_coinc = df_about_coinc[ (df_about_coinc['ns'] != 0) ]
# Suformuojame atitinkamą failų vardų sąrašą
lst_filenames_rpeaks_not_coinc = list(df_rpeaks_not_coinc['filename'].astype(str))
print(f"Failų vardų sąrašas su nesutampančiais rpeaks: len = {len(lst_filenames_rpeaks_not_coinc)}")

# Surenkame eilutes, kurios atitinka sutampančius rpeak
# naudojame požymius Flagged as Deleted == No ir Has Deviated == No
df_rpeaks_coinc = df_about_coinc[ (df_about_coinc['ns'] == 0) ]
# Suformuojame atitinkamą failų vardų sąrašą
lst_filenames_rpeaks_coinc = list(df_rpeaks_coinc['filename'].astype(str))
print(f"Failų vardų sąrašas su sutampančiais rpeaks: len = {len(lst_filenames_rpeaks_coinc)}")


# 2. Nuskaitome Zive įrašų talpykloje laikomų įrašų sąrašą, skirtą klasifikatorių konstravimui
rec_list_1087 = 'list_1087.json'
print("\nInformacija iš failo:", rec_list_1087)
file_path = Path(db_path, rec_list_1087)
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())
df_list = pd.json_normalize(data, record_path =['data'])
lst_file_names_for_samples = list(df_list["file_name"].astype(str))
print(f"Faile yra atrinktų klasifikatorių konstravimui įrašų sąrašas. Jų kiekis: {len(lst_file_names_for_samples)}")
print("Tik iš šių įrašų filtruojant bus formuojami tolesni sąrašai") 
# print("\nFile_names_for_samples:", lst_file_names_for_samples[:10])

# Filtruojame failų vardų sąrašą, įtraukdami tik tuos, kurie yra file_names_for_samples
s = set(lst_file_names_for_samples)
lst_filenames_rpeaks_not_coinc_flt = [x for x in lst_filenames_rpeaks_not_coinc if x  in s]
print(f"Filtruotų failų vardų sąrašas su nesutampančiais rpeaks: len = {len(lst_filenames_rpeaks_not_coinc_flt)}")
# print(f"Pirmi 10: {lst_filenames_rpeaks_not_coinc_flt[:10]}")

# Filtruojame failų vardų sąrašą, įtraukdami tik tuos, kurie yra file_names_for_samples
s = set(lst_file_names_for_samples)
lst_filenames_rpeaks_coinc_flt = [x for x in lst_filenames_rpeaks_coinc if x  in s]
print(f"Filtruotų failų vardų sąrašas su sutampančiais rpeaks: len = {len(lst_filenames_rpeaks_coinc_flt)}")
# print(f"Pirmi 10: {lst_filenames_rpeaks_coinc_flt[:10]}")

# Surandame nepatekusių į klasifikatorių konstravimui skirtų įrašų sąrašą.
# Išvedame jų įrašus
s = set(lst_file_names_for_samples)
lst_file_names_not_for_samples = [x for x in lst_file_names_all if x  not in s]
df_nestmp = df_about_coinc[df_about_coinc['filename'].isin(lst_file_names_not_for_samples)]
print(f"\nNepatekusių į klasifikatorių konstravimui skirtų įrašų sąrašas. Sąrašo ilgis: {len(df_nestmp)}")
print(f"Išvedama 20 įrašų iš rušiuoto sąrašo su daugiausia nesutampančių rpeaks")
# print(df_nestmp)
# Suriūšuojame spausdinimui
df_nestmp_sort = df_nestmp.sort_values(by=['ns'], ascending=False)
print()
print(df_nestmp_sort.head(20))

# 3. Nuskaitome failą su statistika apie koreguotus rpeaks 'forensics.csv, gautus iš Algirdo'
#    Iš jo suformuojame failus su EKG failų vardais: vieną, kur rpeaks koreguoti, kitą - kur rpeaks nejudinti  
filename_about_rpeaks_coreg = 'forensics_org.csv'
print("\nInformacija iš failo:", filename_about_rpeaks_coreg )
file_path = Path(rec_dir, filename_about_rpeaks_coreg)
df_coreg = pd.read_csv(file_path, dtype={"Filename":"string"})
# print(df_coreg.head())

# Išmetame eilutes su informacija apie senus duomenis
drops = 0
for index, row in df_coreg.iterrows():
    filename = row['Filename']
    suffix = Path(filename).suffix
    if suffix == '.DAT':
        df_coreg.drop(index, inplace = True)
        drops += 1
print(f"Išmestų eilučių su extensions .DAT: {drops}")        
# print(df_coreg.head())
# print(len(df_coreg))

# Surenkame visus nepasikartojančius failų vardus iš Algirdo failo
gr = df_coreg.groupby("Filename")
# print(gr.ngroups)
lst_filenames_rpeaks_coreg_all = [key for key, _ in gr]
# print("\nFailai su koreguotais rpeaks (pirmi 10): ", lst_filenames_rpeaks_coreg_all[:10])

# lst_filenames_rpeaks_all = df_coreg['Filename']
# print(f"\nViso įrašų faile lst_filenames_rpeaks_all: {len(lst_filenames_rpeaks_all)}")

# Surenkame eilutes, kurios atitinka koreguotą rpeak
# naudojame požymius Flagged as Deleted == Yes arba Has Deviated == Yes
df_rpeaks_coreg = df_coreg[ (df_coreg['Flagged as Deleted'] == 'Yes') | (df_coreg['Has Deviated'] == 'Yes')]
# print(df_rpeaks_coreg.head())
print(f"Įrašų su koreguotais rpeak faile {filename_about_rpeaks_coreg} rasta: {len(df_rpeaks_coreg)}")
print(f"Viso nepasikartojančių failų vardų sąrašo ilgis: {len(lst_filenames_rpeaks_coreg_all)}")

# Suformuojame failų su koreguotais rpeaks vardų sąrašą
gr = df_rpeaks_coreg.groupby("Filename")
# print(gr.ngroups)
lst_filenames_rpeaks_coreg = [key for key, _ in gr]
# print("\nFailai su koreguotais rpeaks (pirmi 10): ", lst_filenames_rpeaks_coreg[:10])
# print(f"Failų sąrašo ilgis: {len(lst_filenames_rpeaks_coreg)}")

# Surenkame eilutes, kurios atitinka nekoreguotą rpeak
# naudojame požymius Flagged as Deleted == No ir Has Deviated == No
df_rpeaks_not_coreg = df_coreg[ (df_coreg['Flagged as Deleted'] == 'No') & (df_coreg['Has Deviated'] == 'No')]
if (df_rpeaks_not_coreg.empty):
    print(f"Įrašų su nekoreguotais rpeak faile {filename_about_rpeaks_coreg} nerasta")
else:
    print(f"\nĮrašų su nekoreguotais rpeak faile {filename_about_rpeaks_coreg} rasta: {len(df_rpeaks_not_coreg)}")  
    print(df_rpeaks_not_coreg.head())

# Patikriname, kiek Algirdo sutvarkytame faile yra įrašų su failų vardais iš file_names_for_samples
s = set(lst_file_names_for_samples)
all_ = [x for x in lst_filenames_rpeaks_coreg_all if x in s]
# print(f"Koreguotų failų sąraše yra rasta {len(all_)} vardų iš file_names_for_samples")

# # Surandame faile lst_filenames_rpeaks_coreg_all failų vardus, kurie yra ir lst_filenames_about_rpeaks 
# s = set(lst_filenames_about_rpeaks)
# all_ = [x for x in lst_filenames_rpeaks_coreg_all if x not in s]
# print("all_:", len(all_))

# s = set(lst_filenames_rpeaks_coreg_all)
# all_ = [x for x in lst_filenames_about_rpeaks if x not in s]
# print("all_:", len(all_))



print("\nSULYGINIMAS NAUDOJANT VISUS ANOTUOTUS ĮRAŠUS")

s = set(lst_filenames_rpeaks_coreg)
nestmp = [x for x in lst_filenames_rpeaks_coinc if x not in s]
stmp = [x for x in lst_filenames_rpeaks_coinc if x in s]
print(f"\nLyginame lst_filenames_rpeaks_coinc: {len(lst_filenames_rpeaks_coinc)} su lst_filenames_rpeaks_coreg: {len(lst_filenames_rpeaks_coreg)}")
print(f"Nesutampa: {len(nestmp)} Sutampa: {len(stmp)} Iš viso: {len(nestmp)+len(stmp)}")
if (len(stmp) != 0):
    print("Failai su sutampančiais failų vardais (pirmi 10): ", stmp[:10])

nestmp = [x for x in lst_filenames_rpeaks_not_coinc if x not in s]
stmp = [x for x in lst_filenames_rpeaks_not_coinc if x in s]
print(f"\nLyginame lst_filenames_rpeaks_not_coinc: {len(lst_filenames_rpeaks_not_coinc)} su lst_filenames_rpeaks_coreg: {len(lst_filenames_rpeaks_coreg)}")
print(f"Nesutampa: {len(nestmp)} Sutampa: {len(stmp)} Iš viso: {len(nestmp)+len(stmp)}")
# print("Failai su nesutampančiais failų vardais (pirmi 10): ", nestmp[:10])

# 4.1 Suformuojame nesutampančių sąrašą
file_path = Path(rec_dir, filename_about_rpeaks_coinc) # Failas apie rpeaks sutapimą
df = pd.read_csv(file_path, dtype={"ind":int, "filename":"string"})
df['filename'] = df['filename'].astype(str)

df_stmp = df[df['filename'].isin(stmp)]
# print("stmp:", stmp)
print(f"\nKoreguotų įrašų, galimai turinčių deformuotų rpeaks, sąrašas. Sąrašo ilgis: {len(df_stmp)}")
# Suriūšuojame spausdinimui
df_stmp_sort = df_stmp.sort_values(by=['ns'], ascending=True)
# print()
# print(df_stmp_sort)

df_nestmp = df[df['filename'].isin(nestmp)]
print(f"Nekoreguotų įrašų, galimai turinčių deformuotų rpeaks, sąrašas. Sąrašo ilgis: {len(df_nestmp)}")
# print(df_nestmp)
# Suriūšuojame spausdinimui
df_nestmp_sort = df_nestmp.sort_values(by=['ns'], ascending=True)
print()
print(df_nestmp_sort)

print(f"\nViso patikrintų failų: {len(df_stmp) + len(df_nestmp)} ")
print(f"Koreguotų: {len(df_stmp)}  Nekoreguotų: {len(df_nestmp)} ")


print("\nSULYGINIMAS NAUDOJANT ATRINKTUS ANOTUOTUS ĮRAŠUS KLASIFIKATORIAUS KONSTRAVIMUI")

s = set(lst_filenames_rpeaks_coreg)
nestmp = [x for x in lst_filenames_rpeaks_coinc_flt if x not in s]
stmp = [x for x in lst_filenames_rpeaks_coinc_flt if x in s]
print(f"\nLyginame lst_filenames_rpeaks_coinc_flt: {len(lst_filenames_rpeaks_coinc_flt)} su lst_filenames_rpeaks_coreg: {len(lst_filenames_rpeaks_coreg)}")
print(f"Nesutampa: {len(nestmp)} Sutampa: {len(stmp)} Iš viso: {len(nestmp)+len(stmp)}")
if (len(stmp) != 0):
    print("Failai su sutampančiais failų vardais (pirmi 10): ", stmp[:10])

nestmp = [x for x in lst_filenames_rpeaks_not_coinc_flt if x not in s]
stmp = [x for x in lst_filenames_rpeaks_not_coinc_flt if x in s]
print(f"\nLyginame lst_filenames_rpeaks_not_coinc_flt: {len(lst_filenames_rpeaks_not_coinc_flt)} su lst_filenames_rpeaks_coreg: {len(lst_filenames_rpeaks_coreg)}")
print(f"Nesutampa: {len(nestmp)} Sutampa: {len(stmp)} Iš viso: {len(nestmp)+len(stmp)}")
# print("Failai su nesutampančiais failų vardais (pirmi 10): ", nestmp[:10])

# 4.2 Suformuojame nesutampančių sąrašą
file_path = Path(rec_dir, filename_about_rpeaks_coinc) # Failas apie rpeaks sutapimą
df = pd.read_csv(file_path, dtype={"ind":int, "filename":"string"})
df['filename'] = df['filename'].astype(str)

df_stmp = df[df['filename'].isin(stmp)]
# print("stmp:", stmp)
print(f"\nKoreguotų įrašų, galimai turinčių deformuotų rpeaks, sąrašas. Sąrašo ilgis: {len(df_stmp)}")
# Suriūšuojame spausdinimui
df_stmp_sort = df_stmp.sort_values(by=['ns'], ascending=True)
print()
print(df_stmp_sort)

# Įrašome df_stmp_sort masyvą į rec_dir aplanką
file_path = Path(rec_dir, 'KoreguotuRpeaksFailuVarduSarasas.csv')
df_stmp_sort.to_csv(file_path)
print("\nKoreguotų Rpeaks failų vardų sąrašas įrašytas į ", file_path, "\n" )


df_nestmp = df[df['filename'].isin(nestmp)]
print(f"\nNekoreguotų įrašų, galimai turinčių deformuotų rpeaks, sąrašas. Sąrašo ilgis: {len(df_nestmp)}")
# print(df_nestmp)
# Suriūšuojame spausdinimui
df_nestmp_sort = df_nestmp.sort_values(by=['ns'], ascending=False)
print()
print(df_nestmp_sort)

print(f"\nViso patikrintų failų: {len(df_stmp) + len(df_nestmp)} ")
print(f"Koreguotų: {len(df_stmp)}  Nekoreguotų: {len(df_nestmp)} ")




OS in my system :  win32

PASIRUOŠIMAS

Informacija iš failo: information_about_rpeaks_2552.csv
Įrašų faile information_about_rpeaks_2552.csv: 2552
Faile yra visų anotuotų įrašų sąrašas. Jų kiekis: 2552
Failų vardų sąrašas su nesutampančiais rpeaks: len = 621
Failų vardų sąrašas su sutampančiais rpeaks: len = 1931

Informacija iš failo: list_1087.json
Faile yra atrinktų klasifikatorių konstravimui įrašų sąrašas. Jų kiekis: 1087
Tik iš šių įrašų filtruojant bus formuojami tolesni sąrašai
Filtruotų failų vardų sąrašas su nesutampančiais rpeaks: len = 332
Filtruotų failų vardų sąrašas su sutampančiais rpeaks: len = 755

Nepatekusių į klasifikatorių konstravimui skirtų įrašų sąrašas. Sąrašo ilgis: 1465
Išvedama 20 įrašų iš rušiuoto sąrašo su daugiausia nesutampančių rpeaks

      Unnamed: 0   ind     filename                    userId                     recId    ns  E
1934        1934  1934  1637271.521  61a885ec5cd54753a6a4e96e  61a8881ae8f900517cc6e2ad  1136  0
1937        1937  1937  1